In [3]:
import os
import pandas as pd
import pickle
import subprocess
os.chdir("/Users/yogi/Projects/genefeatures/python")
import genefeatures.gtf_tools as gt
import genefeatures.fasta_tools as ft
from intervaltree import Interval, IntervalTree
from genefeatures.sequence_index import SequenceIndex as SI
from genefeatures.mutation_handler import MutationHandler as MH
from genefeatures.variation_parser import SequenceVariationParser as SVP
from Bio.Seq import Seq, reverse_complement


os.listdir("../../references")

['kras_plus_10kb.fa.fai',
 'Homo_sapiens.GRCh38.dna.primary_assembly.fa.fai',
 'Homo_sapiens.GRCh38.112.gtf',
 'hs_grch38_trunc.gtf',
 'Homo_sapiens.GRCh38.dna.primary_assembly.fa',
 'kras_plus_10kb.fa',
 'test_regions.txt']

In [5]:
%%time
gtf = gt.parse_gtf("../../references/Homo_sapiens.GRCh38.112.gtf")

CPU times: user 43.4 s, sys: 2.13 s, total: 45.6 s
Wall time: 46.4 s


In [ ]:
set(gtf.attribute_index.keys())

In [ ]:
set(gtf.feature_index.keys())

In [ ]:
dep_mt = pd.read_csv("../data/depmap/OmicsSomaticMutations.csv", low_memory=False)
models = pd.read_csv("../data/depmap/OmicsProfiles.csv")
models.head()


In [ ]:
rna = pd.read_csv(
    "../data/depmap/OmicsExpressionRNASeQCGeneCountProfile.csv",
    index_col = "Unnamed: 0"
)
rna.head()

In [ ]:
rna_genes = [i.split(" ")[1].strip("()") for i in rna.columns.to_list()]
gtf_genes = gtf.attribute_index["gene_id"].keys()
both = list(set(rna_genes).intersection(set(gtf_genes)))

In [ ]:
with open("../data/genefeatures/make_gf_fasta_inputs/wild_type.csv", "w") as f:
    for i in both:
        f.write(f"{i},,\n")


In [ ]:
itrees = [""] * len(both)
for i, g in enumerate(both):
    records = gtf.query({"attributes": {"gene_name": g}}, return_records = True)
    itrees[i] = gt.records_to_interval_tree(records)

In [ ]:
with open("../references/test_regions.txt", "w") as f:
    for tree in itrees:
        f.write(
            f"{list(tree.all_intervals)[0].data['seqname']}:{tree.begin()}-{tree.end()}\n"
        )

In [ ]:
    
command = "samtools faidx --fai-idx " \
    "../references/Homo_sapiens.GRCh38.dna.primary_assembly.fa.fai " \
    "../references/Homo_sapiens.GRCh38.dna.primary_assembly.fa " \
    f"-r ../references/test_regions.txt"

result = subprocess.run(
    command,
    shell=True,
    capture_output=True,
    text=True
    )

In [ ]:
canon = gtf.query({"attributes": {"tag": "Ensembl_canonical"}})

In [ ]:
canon[100]

In [6]:
foo = gtf.query(
        {"attributes": {"gene_id": ["ENSG00000133703", "ENSG00000141510"], "tag": "MANE_Select"}}
)

In [8]:
len(foo)

42

# gtf edits needed
* filter records with no start and end
    * remove_empty_fields method already implemented and is not ridiculously slow
* skip records where start == end when making interval trees
* getting sequences for every gene in transcriptome (36k genes) takes roughly 11 min
    * speed up: reduce size of transcriptome - diminishing returns, can really only get to 23k
    * keep seqname at top level of whatever's calling extract_sequences to avoid type transformation
    * two above are not what makes this slow
    * need to call faidx in batch using region file
* convert entire gtf to list of GeneFeature or ndarray of GeneFeature
* query strings would be nice to have
* might need option to skip SeqIndex generation when no mutations are needed
* Or for larger workflow make indices in the beginnning and copy/mutate for each model

worklow for making wild type fasta
1. get list of genes and primary transcripts
2. generate data structure that holds primary transcripts/SeqIndex
4. read dna sequence for each and write to fasta

